**Alunos:** 

Breno Pontes Da Costa - 114036496

Xiao Yong Kong - 114176987

In [ ]:
# carregando o drive
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: ignored

In [ ]:
# Carregando Bibliotecas
import pandas as pd
from sklearn.tree import DecisionTreeClassifier # Importa o classificador de árvore de decisão
from sklearn import metrics #Importa métrica para calcular acurácia - módulo do scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.tree import export_graphviz
from sklearn.externals.six import StringIO  
from IPython.display import Image  
import pydotplus

Usaremos como base de dados o arquivo "exemplo_IA.csv" com os dados usados no exemplo da "aula 9 - video 3 - aprendizado de máquina.pdf".

In [ ]:
# Nome dos atributos
col_names = ['Outlook','Temperature','Humidity','Wind','PlayTennis']

# carregando a base de dados a ser usada
exemplo = pd.read_csv("/content/drive/exemplo_IA.csv", header=None, names=col_names)

In [ ]:
# podemos ver os primeiros elementos do arquivo
exemplo.head()

In [ ]:
# definir o conceito alvo e as features usadas

feature_cols = ['Outlook','Temperature','Humidity','Wind'] # features (atributos) que serão usadas no aprendizado

X = exemplo[feature_cols] # selecionamos as colunas correspondentes aos atributos que serão usados
y = exemplo.PlayTennis # conceito que queremos aprender. Selecionamos a coluna com a classificação das instâncias

Os dados do nosso exemplo são categóricos e o classificador de árvore de decisão do scikit não funciona para este tipo de dado. Precisamos preparar os dados antes construir nosso classificador. Para isso vamos usar a função do pandas "pd.get_dummies" que converte os atributos categóricos em atributos com valores 0 e 1.
Por exemplo, o atributo Humidity que pode receber os valores High e Normal, passa a ser representado pelos atributos Humidity_High e Humidity_Normal, que podem receber os valores 0 e 1.

In [ ]:
Xone_hot_data = pd.get_dummies(X[['Outlook','Temperature','Humidity','Wind']]) #transformando o dataset
Xone_hot_data.head()

Vamos criar agora o nosso classificador de árvore de decisão, usando como função para seleção do atributo a entropia:

In [ ]:
def classifier(X_train, y_train, criteria):
  # Criação do classificador de árvore de decisão 
  clf = DecisionTreeClassifier(criterion=criteria)

  # Usamos o método fit para construir o classificador a partir do nosso conjunto de treinamento
  clf = clf.fit(X_train, y_train)
  return clf

Podemos visualizar a árvore de decisão gerada usando o pacote graphviz:
https://graphviz.org/
https://pypi.org/project/graphviz/

In [ ]:
def generate_tree(clf, feature_names=Xone_hot_data.columns):
  # tem que usar feature_names = one_hot_data.columns pois feature_names = feature_cols tem menos atributos
  # pois o one-hot acrescenta mais
  dot_data = StringIO()
  export_graphviz(clf, out_file=dot_data,  
                  filled=True, rounded=True,
                  special_characters=True,feature_names=feature_names,class_names=['No','Yes'])
  graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
  return graph


Podemos medir a acurácia testando quantas instâncias são classificadas corretamente. Observe que usamos todas as instâncias disponíveis no treinamento. A acurácia obtida foi 1, indicando que classificamos corretamente todos os exemplos de nosso conjunto.

In [ ]:
def prediction_accuracy(clf, X_test, y_test):
  # Usando modelo para classificar os dados que temos a disposição
  y_pred = clf.predict(X_test)

  # Medida de acuracia, que indica quantas instâncias são corretamente classificadas
  return metrics.accuracy_score(y_test, y_pred)

Sem divisão em grupos de treino e teste


In [ ]:
criteria = "entropy"
clf_entropy_100 = classifier(Xone_hot_data, y, criteria)
graph_entropy_100 = generate_tree(clf_entropy_100)

graph_entropy_100.write_png('arvore-100.png')
Image(graph_entropy_100.create_png())

In [ ]:
prediction_entropy_100 = prediction_accuracy(clf_entropy_100, Xone_hot_data, y)
print('Acurácia: ', prediction_entropy_100)

Dividindo em 60% treinamento e 40% teste:


In [ ]:
X_train_60, X_test_60, y_train_60, y_test_60 = train_test_split(Xone_hot_data, y, test_size=0.40, random_state=1)

criteria = "entropy"
clf_entropy_60 = classifier(X_train_60, y_train_60, criteria)
graph_entropy_60 = generate_tree(clf_entropy_60)

graph_entropy_60.write_png('arvore-60-40.png')
Image(graph_entropy_60.create_png())

In [ ]:
prediction_test_entropy_60 = prediction_accuracy(clf_entropy_60, X_test_60, y_test_60)
print('Acurácia teste: ', prediction_test_entropy_60)

In [ ]:
prediction_train_entropy_60 = prediction_accuracy(clf_entropy_60, X_train_60, y_train_60)
print('Acurácia treino: ', prediction_train_entropy_60)

In [ ]:
prediction_full_60 = prediction_accuracy(clf_entropy_60, Xone_hot_data, y) #passando a base completa (treino + teste) para ser testada
print('Acurácia completo: ', prediction_full_60)

Dividindo em 80% treino e 20% teste:


In [ ]:
X_train_80, X_test_80, y_train_80, y_test_80 = train_test_split(Xone_hot_data, y, test_size=0.20, random_state=1)
criteria = "entropy"
clf_entropy_80 = classifier(X_train_80, y_train_80, criteria)
graph_entropy_80_20 = generate_tree(clf_entropy_80)

graph_entropy_80_20.write_png('arvore-80-20.png')
Image(graph_entropy_80_20.create_png())

In [ ]:
prediction_test_entropy_80 = prediction_accuracy(clf_entropy_80, X_test_80, y_test_80)
print('Acurácia teste: ', prediction_test_entropy_80)

In [ ]:
prediction_train_entropy_80 = prediction_accuracy(clf_entropy_80, X_train_80, y_train_80)
print('Acurácia treino: ', prediction_train_entropy_80)

In [ ]:
prediction_full_entropy_80 = prediction_accuracy(clf_entropy_80, Xone_hot_data, y) #passando a base completa (treino + teste) para ser testada
print('Acurácia completo: ', prediction_full_entropy_80)

Repetindo os casos testados para Gini:

Utilizando 100% da base para treino


In [ ]:
criteria = "gini"
clf_gini_100 = classifier(Xone_hot_data, y, criteria)
graph_gini_100 = generate_tree(clf_gini_100)

graph_gini_100.write_png('arvore-100-gini.png')
Image(graph_gini_100.create_png())

In [ ]:
prediction_gini_100 = prediction_accuracy(clf_gini_100, Xone_hot_data, y) #passando a base completa (treino + teste) para ser testada
print('Acurácia: ', prediction_gini_100)

Utilizando 60% da base para treino, e 40% para teste


In [ ]:
criteria = "gini"
clf_test_gini_60 = classifier(X_train_60, y_train_60, criteria)
graph_gini_60_40 = generate_tree(clf_test_gini_60)

graph_gini_60_40.write_png('arvore-60-40-gini.png')
Image(graph_gini_60_40.create_png())

In [ ]:
prediction_test_gini_60 = prediction_accuracy(clf_test_gini_60, X_test_60, y_test_60)
print('Acurácia teste: ', prediction_test_gini_60)

Dividindo a base em 80% para treino, e 20% para teste

In [ ]:
criteria = "gini"
clf_gini_80 = classifier(X_train_80, y_train_80, criteria)
graph_gini_80_20 = generate_tree(clf_gini_80)

graph_gini_80_20.write_png('arvore-80-20-gini.png')
Image(graph_gini_80_20.create_png())

In [ ]:
prediction_test_gini_80 = prediction_accuracy(clf_gini_80, X_test_80, y_test_80)
print('Acurácia teste: ', prediction_test_gini_80)

Utilizando K-fold

In [ ]:
cv = KFold(n_splits=8, random_state=1, shuffle=True)
# for train, test in cv.split(Xone_hot_data):
#   print("%s %s" % (train, test))

model = DecisionTreeClassifier(criterion='entropy')
scores = cross_validate(model, Xone_hot_data, y, scoring=['accuracy'], return_train_score=True, cv=cv)
#scores = cross_val_score(model, Xone_hot_data, y, scoring='accuracy', cv=cv)
print('Accuracy Test: %.3f (%.3f)' % (scores['test_accuracy'].mean(), scores['test_accuracy'].std()))
print('Accuracy Train: %.3f (%.3f)' % (scores['train_accuracy'].mean(), scores['train_accuracy'].std()))

# Árvore de Decisão - Diabetes


In [ ]:
# carregando a base de dados a ser usada
exemplo_diabetes = pd.read_csv("/content/drive/diabetes-trabalho.csv")

In [ ]:
exemplo_diabetes.head()


In [ ]:
# definir o conceito alvo e as features usadas

feature_cols_diabetes = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']
X_diabetes = exemplo_diabetes[feature_cols_diabetes] # selecionamos as colunas correspondentes aos atributos que serão usados
y_diabetes = exemplo_diabetes.Outcome # conceito que queremos aprender. Selecionamos a coluna com a classificação das instâncias

Gerando a árvore de decisão


In [ ]:
criteria = "entropy"
clf_diabetes = classifier(X_diabetes, y_diabetes, criteria)
graph_diabetes = generate_tree(clf_diabetes, X_diabetes.columns)

graph_diabetes.write_png('arvore-diabetes.png')
Image(graph_diabetes.create_png())

Testando a acurácia do classificador gerado

In [ ]:
X_train_80_diabetes, X_test_80_diabetes, y_train_80_diabetes, y_test_80_diabetes = train_test_split(X_diabetes, y_diabetes, test_size=0.20, random_state=1)
criteria = "entropy"
clf_entropy_80_diabetes = classifier(X_train_80_diabetes, y_train_80_diabetes, criteria)
graph_entropy_80_diabetes = generate_tree(clf_entropy_80_diabetes, X_train_80_diabetes.columns)

graph_entropy_80_diabetes.write_png('arvore-80-diabetes.png')
Image(graph_entropy_80_diabetes.create_png())

In [ ]:
prediction_test_diabetes = prediction_accuracy(clf_entropy_80_diabetes, X_test_80_diabetes, y_test_80_diabetes)
print('Acurácia teste: ', prediction_test_diabetes)

In [ ]:
prediction_train_diabetes = prediction_accuracy(clf_diabetes, X_diabetes, y_diabetes)
print('Acurácia teste: ', prediction_train_diabetes)